Recurrent Neural Network 
=================================================

In [1]:
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.ops import rnn, rnn_cell

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
hm_epochs = 10
n_classes = 10
batch_size = 128
chunk_size = 28
n_chunks = 28
rnn_size = 128

In [4]:
x = tf.placeholder('float', [None, n_chunks,chunk_size], name = 'x_')
y = tf.placeholder('float', name = 'y_')

In [5]:
layer = {'weights':tf.Variable(tf.random_normal([rnn_size,n_classes]), name = 'w1'),
             'biases':tf.Variable(tf.random_normal([n_classes]), name = 'b1')}
lstm_cell = rnn_cell.BasicLSTMCell(rnn_size,state_is_tuple=True)
sess=tf.Session()
saver = tf.train.Saver()

In [6]:
def recurrent_neural_network(x):
    x = tf.transpose(x, [1,0,2], name = 'trp')
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, n_chunks, 0, name = 'x')
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32 )
    output = tf.matmul(outputs[-1],layer['weights'], name = 'output') + layer['biases']

    return output

In [7]:
def train_neural_network(x):
    prediction = recurrent_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction), name='cost' )
    optimizer = tf.train.AdamOptimizer(name='beta_power').minimize(cost)
    
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size,n_chunks,chunk_size))       
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape((-1, n_chunks, chunk_size)), y:mnist.test.labels}))

In [8]:
train_neural_network(x)

Epoch 0 completed out of 10 loss: 205.559702717
Epoch 1 completed out of 10 loss: 55.2422834132
Epoch 2 completed out of 10 loss: 37.9674362652
Epoch 3 completed out of 10 loss: 28.5930123888
Epoch 4 completed out of 10 loss: 23.4807348601
Epoch 5 completed out of 10 loss: 19.5860731995
Epoch 6 completed out of 10 loss: 15.6156576817
Epoch 7 completed out of 10 loss: 14.5956028575
Epoch 8 completed out of 10 loss: 13.0615695803
Epoch 9 completed out of 10 loss: 12.0652420297
Accuracy: 0.9864
